In [1]:
import pandas as pd

In [5]:
%pwd

'/content'

In [16]:
!head train.csv

id,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,Darrell Lucus,"House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) 
With apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. 
As we now know, Comey notified the Republican chairmen and D

In [17]:
df=pd.read_csv('train.csv')

In [18]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [19]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [20]:
###Drop Nan Values
df=df.dropna()

In [21]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [22]:
## Get the Dependent features
y=df['label']

In [23]:
X.shape

(18285, 4)

In [24]:
y.shape

(18285,)

In [25]:
import tensorflow as tf

In [27]:
tf.__version__

'2.15.0'

In [28]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [30]:
messages=X.copy()

In [31]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [32]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [64]:
messages.reset_index(inplace=True)

In [65]:
import nltk
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [36]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [73]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


In [74]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [75]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [76]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[4771, 2248, 4489, 3043, 3657, 1438, 2156, 4329, 1296, 2939],
 [3663, 593, 527, 2690, 4400, 2485, 774],
 [2241, 3807, 3798, 3768],
 [2396, 1157, 115, 1171, 1698, 1801],
 [4413, 4400, 3952, 769, 435, 3125, 4400, 2380, 3536, 2597],
 [617,
  4637,
  15,
  3913,
  4771,
  404,
  722,
  4536,
  4315,
  3940,
  1824,
  4782,
  3552,
  3387,
  774],
 [1192, 1398, 2360, 4492, 1948, 4347, 4127, 6, 4279, 4186, 3946],
 [1249, 609, 4610, 2866, 3700, 2361, 404, 4180, 4279, 4186, 3946],
 [4366, 2082, 2203, 1024, 3876, 2115, 155, 2657, 404, 94],
 [2179, 3565, 4753, 633, 4578, 2249, 3508, 2146],
 [3861, 2142, 1611, 1163, 4709, 940, 1701, 1673, 688, 689, 442],
 [1171, 791, 3657, 2115, 404, 3700],
 [4677, 4758, 680, 4779, 4810, 2425, 4769, 311, 1969],
 [575, 2889, 3909, 2707, 3506, 2508, 3716, 4279, 4186, 3946],
 [4837, 4020, 1887, 4598, 2446, 4279, 4186, 3946],
 [3187, 362, 673, 4418, 584, 535, 1116, 3891, 579, 2424],
 [4218, 2137, 593],
 [1566, 4489, 4333, 3179, 404, 45, 1228, 774],
 [2319, 4167, 527

In [77]:
onehot_repr[1]

[3663, 593, 527, 2690, 4400, 2485, 774]

### Embedding Representation


In [79]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4329 1296 2939]
 [   0    0    0 ... 4400 2485  774]
 [   0    0    0 ... 3807 3798 3768]
 ...
 [   0    0    0 ... 4279 4186 3946]
 [   0    0    0 ... 3234 4250 4563]
 [   0    0    0 ... 2548 3075 1913]]


In [80]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 3663,  593,  527, 2690, 4400, 2485,  774], dtype=int32)

In [81]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [82]:
len(embedded_docs),y.shape

(18285, (18285,))

In [83]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [84]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [85]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training


In [99]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 46ms/step - loss: 0.3470 - accuracy: 0.8331 - val_loss: 0.1938 - val_accuracy: 0.9181
Epoch 2/10
192/192 [==============================] - 2s 13ms/step - loss: 0.1515 - accuracy: 0.9400 - val_loss: 0.1891 - val_accuracy: 0.9201
Epoch 3/10
192/192 [==============================] - 2s 11ms/step - loss: 0.1097 - accuracy: 0.9587 - val_loss: 0.1988 - val_accuracy: 0.9137
Epoch 4/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0847 - accuracy: 0.9691 - val_loss: 0.2371 - val_accuracy: 0.9167
Epoch 5/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0631 - accuracy: 0.9776 - val_loss: 0.2452 - val_accuracy: 0.9138
Epoch 6/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0455 - accuracy: 0.9842 - val_loss: 0.2986 - val_accuracy: 0.9135
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0350 - accuracy: 0.9882 - val_loss: 0.3304 - val_accuracy: 0.9180

### Adding Dropout

In [87]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy


In [100]:
y_pred=model.predict(X_test)

189/189 [==============================] - 0s 2ms/step


In [101]:
y_pred=np.where(y_pred > 0.5, 1,0)

In [102]:
from sklearn.metrics import confusion_matrix

In [103]:
confusion_matrix(y_test,y_pred)

array([[3112,  307],
       [ 212, 2404]])

In [104]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9140016570008285

In [105]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92      3419
           1       0.89      0.92      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

